In [2]:
# 將財產分配 load 程式
import pandas as pd
import numpy as np
import time
from datetime import date
import datetime
import re
import os


def parse_btsid(value):
    pattern = 'R\d{4,6}|\d{4,7}[uUlLnNgG]'
    m = re.search(pattern, value)
    if m and m.group(0):
        return m.group(0)
    else:
        return '沒填寫'

def detect_star(value):
    pattern = '([A-Za-z]{4,6})\*2|[A-Za-z0-9,;\[\]+]{4,24}'  # 還必須考慮,;[+]+等
    m = re.search(pattern, str(value))
    if m and m.group(1):
        return m.group(1) + ' ' + m.group(1)
    elif m and m.group(0):
        return m.group(0) 
    
def find_earlier_HW_Link():
    list1=['','']
    path= './data_HW_Link'
    date_list = os.listdir(path)
    n=0
    now1= date.today() 
    for i in range(30):   
        date_name_4G = '4G HW_Link_{}E.xlsm'.format(now1).replace('-','')
        date_name_5G = '5G HW_Link_{}E.xlsm'.format(now1).replace('-','')
        if date_name_4G in date_list:
            if list1[0] =='':
                list1[0] = date_name_4G
                n=n+1
        if date_name_5G in date_list:
            if list1[1] =='':
                list1[1] = date_name_5G
                n=n+1
        if n==2:
            break
        else:
            now1 = now1 - datetime.timedelta(days=1)
    return(list1)

def assets_duplicate(data):
    total_len=len(data)
    del_index=[]
    for i in range(total_len):
        if len(data[i])>1:
            res = data[i]
            del_index.append(res)
            output=[res[i:i + 1] for i in range(0, len(res), 1)]
            for j in range(len(output)):
                data.append(output[j])     

    for i in del_index:
        data.remove(i)
    data.sort()  
    return data

def find_last_record():
    list1=['']
    path= './results(analysis)'
    date_list = os.listdir(path)
    now= date.today() 
    now1 = now - datetime.timedelta(days=1)
    for i in range(30):   
        antenna_df = 'bts_circuitboard_switch({}).xlsx'.format(now1)
        if antenna_df in date_list:
            if list1[0] =='':
                list1[0] = antenna_df
            break
        else:
            now1 = now1 - datetime.timedelta(days=1)
    return(list1)

# --------讀取 assets 基地台資料庫 (assets_df) ----------- 
today = date.today() 
file1_name = 'bts_figure({}).xlsx'.format(today)
file1_path = "./results(analysis)/{}".format(file1_name)
assets_df = pd.read_excel(file1_path, sheet_name = '基地台設備')
# ---------寫入資料庫  ---------------------------------
today = date.today() 
writer =pd.ExcelWriter('./results(analysis)/bts_circuitboard_switch({}).xlsx'.format(today))   

#---------調整 assets_df 格式 -----------
# 將 3G「FRGY」 電路板放入 4G 5G 財產中
assets_FRGY_df = assets_df[assets_df['財產名稱'].isin(['3G行動電話收發訊系統']) & 
                             assets_df['型式/號'].str.contains("FRGY") ].copy()
assets_FRGY_df['型式/號'] = assets_FRGY_df['型式/號'].str.replace("Flexi-RRH","").str.replace('(','').str.replace(')','')

# index1 = assets_FRGY_df.loc[assets_FRGY_df['編號'].str.contains("U")].index   
# assets_FRGY_df.loc[index1 ,'設備狀態'] = '備援/備用' 

assets_FRGY_to_spare_df  = assets_FRGY_df[~assets_FRGY_df['編號'].str.contains("L")] 
index1 = assets_FRGY_df.loc[~assets_FRGY_df['編號'].str.contains("L")].index   
assets_FRGY_df.drop(index1, inplace = True)


assets_df = assets_df[assets_df['財產名稱'].isin(['4G行動寬頻系統共構設備','4G行動寬頻基地台','5G基地台射頻模組','5G基地台基頻模組'])]
assets_df = pd.concat([assets_df, assets_FRGY_df])
assets_df = assets_df.reset_index(drop = True)

assets_inuse_df = assets_df[(assets_df['設備狀態']=='使用中') & (assets_df['數量']== 1)]

assets_spare_df = assets_df[(assets_df['設備狀態']=='備援/備用') & (assets_df['數量']== 1)]
assets_spare_df = pd.concat([assets_spare_df,assets_FRGY_to_spare_df]) 
assets_spare_df  = assets_spare_df.reset_index(drop = True)

assets_stop_df = assets_df[(assets_df['設備狀態']=='停用') & (assets_df['數量']== 1)]

assets_inuse_df = assets_inuse_df[['編號','基地台名稱','型式/號','財產編號','異動者']]
assets_inuse_df['型式/號'] = assets_inuse_df['型式/號'].str.replace("AirScale","").str.replace('(','').str.replace(')','').str.replace('_','')
assets_inuse_df = assets_inuse_df.sort_values(by=['編號'])
assets_inuse_df.rename(columns={'基地台名稱':'基地台名(assets)','型式/號':'型式/號(assets)'},inplace =True)
assets_inuse_df.sort_values(by = ['編號','基地台名(assets)','型式/號(assets)'],inplace = True)
assets_inuse_df.to_excel(writer,sheet_name ='assets(使用中)',index=False)

assets_spare_df.to_excel(writer,sheet_name ='assets(備用)',index=False)
assets_stop_df.to_excel(writer,sheet_name ='assets(停用)',index=False)


#---------調整 netact_df 格式 ------------

# 讀取 NetAct 基地台資料庫 (netact_df)
n = 0
all_files = find_earlier_HW_Link() # 使用 def 
for filename in all_files:
    begin_df = pd.read_excel('./data_HW_Link/'+filename,sheet_name = 'Main',header = 5 )
    list_colname = list(begin_df.head())
    begin_df.rename(columns = {list_colname[0]:'編號',list_colname[1]:'基地台名稱',list_colname[4]:'硬體元件',list_colname[5]:'Serial Number'},inplace = True)
    begin_df = begin_df[['編號','基地台名稱','硬體元件','Serial Number']]
    begin_df['編號'] = begin_df['編號'].astype(str)
    begin_df['Serial Number'] = begin_df['Serial Number'].astype(str)
    begin_df['L_column'] = 'L'
    begin_df['N_column'] = 'N'
    if n == 0 :
        begin_df['編號'] = begin_df[['編號','L_column']].apply(''.join, axis=1)
        netact_df = begin_df
        n = n+1
    else:
        begin_df['編號'] = begin_df[['編號','N_column']].apply(''.join, axis=1)
        netact_df = netact_df.append(begin_df)
        
values = {'硬體元件': 'NULL','Serial Number': 'NULL'}
netact_df.fillna(value=values,inplace = True)

netact_df = netact_df.drop(['L_column','N_column'],axis = 1)  
netact_df = netact_df.reset_index(drop=True)

netact_df = netact_df.drop(netact_df.loc[netact_df['硬體元件'].str.contains(pat ='-1')].index)
netact_df = netact_df.drop(netact_df.loc[netact_df['硬體元件'].str.contains(pat ='473764A.102')].index)

netact_df = netact_df.drop(netact_df.loc[~netact_df['Serial Number'].str.contains(pat = '[A-Z]',regex= True)].index)

netact_df = netact_df[~netact_df['硬體元件'].isin(['FR2EB','FR2HB','FWEA_FREA','FWHN_FRHN'])]
netact_df['硬體元件'] = netact_df['硬體元件'].str.replace('ASIB AirScale Common','ASIB').str.replace('(','').str.replace(')','')   
netact_df['硬體元件'] = netact_df['硬體元件'].str.replace('AZQG S4-90M-R1-V2','AZQG').str.replace(' RV4S4-65A-R6','').str.strip()

netact_df = netact_df.dropna(subset =['基地台名稱'])
netact_df = netact_df.reset_index(drop = True)
netact_df.rename(columns={'基地台名稱':'基地台名(NetAct)','硬體元件':'型式/號(NetAct)'},inplace =True)
netact_df.sort_values(by=['編號','型式/號(NetAct)'],inplace = True)
netact_df = netact_df.reset_index(drop = True)
netact_df.to_excel(writer,sheet_name ='NetAct(使用中)',index=False)   


#---------合併 (assets_df) (netact_df) 兩資料庫內容 ------------
full_df = pd.concat([netact_df, assets_inuse_df])
full_df = full_df.reset_index(drop = True)
full_df = full_df[['編號','基地台名(NetAct)','型式/號(NetAct)','型式/號(assets)','財產編號','異動者']]
full_df.sort_values(by = ['編號','型式/號(NetAct)','型式/號(assets)'],inplace = True)
full_df = full_df.reset_index(drop = True)


# --------比較 (assets_df) (netact_df)兩資料庫內容 ----------- 
full_df['型式/號(assets)'] = full_df['型式/號(assets)'].map(detect_star)
full_df['型式/號(assets)'] = full_df['型式/號(assets)'].str.replace(',',' ').str.replace(';',' ').str.replace('[','').str.replace(']','').str.replace('+',' ')
# full_df.to_excel(writer,sheet_name ='基地台設備調整')

bts_id = list(full_df['編號'].unique())
for i in bts_id:
    netact_tmp_df = full_df[(full_df['編號']== i)&(full_df['型式/號(assets)'].isnull())]
    check1 = sorted(list(netact_tmp_df['型式/號(NetAct)'].str.split(' ')))
    assets_tmp_df = full_df[(full_df['編號']== i)&(full_df['型式/號(NetAct)'].isnull())]
    check2 = sorted(list(assets_tmp_df['型式/號(assets)'].str.split(' ')))    
    check2 = assets_duplicate(check2)
    
    # --------去除掉 FBBA & FBBC 電路板的比較----
    check1 = [x for x in check1 if x !=['FBBA']]
    check1 = [x for x in check1 if x !=['FBBC']]
    check2 = [x for x in check2 if x !=['FBBA']]
    check2 = [x for x in check2 if x !=['FBBC']]    
    # ------------------------------------------ 
    
    if str(check1).upper() != str(check2).upper() :
        index1 = full_df.loc[full_df['編號']==i].index   
        full_df.loc[index1 ,'check'] = 'X'
    else:
        index2 = full_df.loc[full_df['編號']==i].index   
        full_df.loc[index2 ,'check'] = 'O'
        
full_df.to_excel(writer,sheet_name ='設備調整',index=False)

# 成績值比較 先選以前值
last_record =find_last_record() # 使用 def Macro
last_record_df = pd.read_excel('./results(analysis)/'+ last_record[0],sheet_name = '完成數',dtype= {'日期': str})

grade_df = full_df.loc[full_df.check=='O'].copy()
grade_df['異動者'].fillna(method='bfill',inplace=True)
grade_df.drop(['基地台名(NetAct)','型式/號(NetAct)','型式/號(assets)','財產編號','check'],axis =1,inplace=True)
grade_df = grade_df.drop_duplicates()
grade1_df = pd.DataFrame(grade_df['異動者'].value_counts()).T
grade1_df['日期']=str(today)
grade1_df.insert(0, '日期', grade1_df.pop('日期'))

result = pd.concat([last_record_df, grade1_df],ignore_index= True)

result.tail(15).to_excel(writer,sheet_name ='完成數',index = False)



# # 關閉寫入檔案
writer.save()



